In [115]:
# -*- coding: utf-8 -*-
"""
[Tom, Lee]
[20034493]
[MMA]
[2021W]
[MMA 865]
[Sunday, October 18, 2020]


Submission to Question [2], Part [1]
"""

import pandas as pd
import numpy as np

df_train = pd.read_csv(r"C:\Users\Tom-T\Google Drive\Queen's MMA\Big Data Analytics - MMA 865\Assignment 1\sentiment_train.csv")

print(df_train.info())
print(df_train.head())

df_test = pd.read_csv(r"C:\Users\Tom-T\Google Drive\Queen's MMA\Big Data Analytics - MMA 865\Assignment 1\sentiment_test.csv")

print(df_train.info())
print(df_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB
None
                                            Sentence  Polarity
0                           Wow... Loved this place.         1
1                                 Crust is not good.         0
2          Not tasty and the texture was just nasty.         0
3  Stopped by during the late May bank holiday of...         1
4  The selection on the menu was great and so wer...         1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB

In [116]:
#EDA

#Class Balance
print(df_train.groupby('Polarity').count())
print(df_test.groupby('Polarity').count())
#Seems good.

          Sentence
Polarity          
0             1213
1             1187
          Sentence
Polarity          
0              287
1              313


In [117]:
#Preprocessing

df_train.head(50)

#Noticed some #NAME excel error values, let's remove them

df_train[df_train['Sentence']=='#NAME?'] #4 instances
df_train = df_train.drop(df_train[df_train.Sentence == '#NAME?'].index) #Removed

In [118]:
#Preprocessing Continued (Combine with above command later)

#Preprocessing via function to allow use in pipeline

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


def my_preprocess(doc):
    
    # Lowercase - caps could be important indicators of strength of polarity, but not directionality. Therefore, I feel 
    # confident lowering all cases
    doc = doc.lower()
        
    # stopwords removal
    split_doc = doc.split()
    stopwords_result = [t for t in split_doc if t not in stopwords.words('english')]
    doc = ' '.join(stopwords_result)
    
    # lemmatization
    doc = WordNetLemmatizer().lemmatize(doc)
    
    return doc


In [119]:
#Test train splitting for cross validation
from sklearn.model_selection import train_test_split

X = df_train['Sentence']
y = df_train['Polarity']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)

In [120]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier

#Attempt 1
vectorizer = CountVectorizer(preprocessor = my_preprocess)
alg = MultinomialNB()

#Attempt 2 - seems like this is the best model at test F1 score of .80
vectorizer = CountVectorizer(preprocessor = my_preprocess, ngram_range=[1,2])
alg = MultinomialNB() #also attempted to tune hyperparameters such as alpha (additive smoothing), but saw no improvement

#Attempt 3
vectorizer = CountVectorizer(preprocessor = my_preprocess, ngram_range=[1,2])
alg = GradientBoostingClassifier()

pipe = Pipeline([('cv', vectorizer), ('alg', alg)])


In [121]:
#Fit model
pipe.fit(X_train, y_train)

Pipeline(steps=[('cv',
                 CountVectorizer(ngram_range=[1, 2],
                                 preprocessor=<function my_preprocess at 0x0000018FA52F9C80>)),
                ('alg', GradientBoostingClassifier())])

In [122]:
#Evaluate model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score

pred_val = pipe.predict(X_val)

print("Confusion matrix:")
print(confusion_matrix(y_val, pred_val))

print("\nF1 Score = {:.5f}".format(f1_score(y_val, pred_val, average="micro")))

print("\nClassification Report:")
print(classification_report(y_val, pred_val))

Confusion matrix:
[[225  18]
 [ 85 152]]

F1 Score = 0.78542

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.93      0.81       243
           1       0.89      0.64      0.75       237

    accuracy                           0.79       480
   macro avg       0.81      0.78      0.78       480
weighted avg       0.81      0.79      0.78       480



In [123]:
#Prediction on Test set, using Attempt #2's model
X = df_test['Sentence']
y = df_test['Polarity']

pred_val = pipe.predict(X)

print("Confusion matrix:")
print(confusion_matrix(y, pred_val))

print("\nF1 Score = {:.5f}".format(f1_score(y, pred_val, average="micro")))

print("\nClassification Report:")
print(classification_report(y, pred_val))

Confusion matrix:
[[270  17]
 [167 146]]

F1 Score = 0.69333

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.94      0.75       287
           1       0.90      0.47      0.61       313

    accuracy                           0.69       600
   macro avg       0.76      0.70      0.68       600
weighted avg       0.76      0.69      0.68       600



In [124]:
#Identify incorrectly classified instances

errors_df = df_test
errors_df['Predictions'] = pred_val
errors_df['Incorrect Flag'] = np.where(errors_df['Predictions'] != errors_df['Polarity'],1,0)

errors_csv = errors_df.loc[errors_df['Incorrect Flag']==1].to_csv(r"C:\Users\Tom-T\Google Drive\Queen's MMA\Big Data Analytics - MMA 865\Assignment 1\errors.csv")